### One of the best things about a Jypyter Notebook in Azure Machine Learning on a compute node is that you have all the requriemetns setup.  Development environment setup can take so long.

In [56]:
import os
import sys
import shutil
import datetime
import time
import azureml
import azureml.core
from azureml.core import Workspace
from azureml.core import Experiment
from azureml.core.container_registry import ContainerRegistry
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core.datastore import Datastore
from azureml.core.runconfig import DataReferenceConfiguration, MpiConfiguration, TensorflowConfiguration
from azureml.train.dnn import TensorFlow
from azureml.core.authentication import InteractiveLoginAuthentication

In [57]:
# Populate these settings
azureSubscriptionID = 'dc6f773e-4b13-4f8b-8d76-f34469246722'    # This is your Azure Subscription ID  -- must exist
azureResourceGroup = 'audioRG'                                  # Azure Resource Group                -- must exist
azureMLWorkSpaceName = 'audioWorkSpace'                         # ML WorkSpace Name                   -- will create if needed
azureMLWorkSpaceLocation = "South Central US"                   # location of Workspace
azureMLClusterName = "cluster3"                                 # the name of the cluster             -- wil create if needed
azureMLVMSize = 'Standard_NC6' # consider 'Standard_D2'         # the VM SKU if creating Cluster
azureMLMazNodes = 1                                             # if you want to scale across nodes
experiment_name = 'KerasAudioExperiment'                        # Expriment Name                      -- will create if needed

# this is the Azure Resource group where the labled audio file are.
azureStorgeAccountName = 'kevinsayazstorage'
azureStorageKeyName = 'YJoRBREMOVEDyDROyTA=='
azureStorageTargetContainer = 'audiolabled'

In [ ]:
# downloading the requried files


In [58]:
# bacause my account has access to several Azure Subscriptions and several Azure AD, specifying the Azure AD (Tenant ID) below:
# InteractiveLoginAuthentication(force=False, tenant_id='da5b399b-aca8-4fee-a37b-f1dcd20717ce', cloud=None)

# if we don't have a workspace, authenticate the user and create one
if azureMLWorkSpaceName not in Workspace.list(subscription_id=azureSubscriptionID):
    ws=Workspace.create(subscription_id=azureSubscriptionID, resource_group=azureResourceGroup, name=azureMLWorkSpaceName, location=azureMLWorkSpaceLocation)
else:
    ws=Workspace.get(azureMLWorkSpaceName, subscription_id=azureSubscriptionID)

# create or use an existing experiment
exp = Experiment(workspace=ws, name=experiment_name)

# register our existing Azure Blob Container with the labled audio files
ds = Datastore.register_azure_blob_container(workspace=ws, datastore_name=azureStorageTargetContainer, container_name=azureStorageTargetContainer,
    account_name=azureStorgeAccountName, account_key=azureStorageKeyName, create_if_not_exists=False)

# create a reference where we mount the DataStore to the container instance
dr = DataReferenceConfiguration(datastore_name=ds.name, path_on_compute='data', mode='mount')

# upload any source files from this Jypyter Notebook
ws.get_default_datastore().upload(src_dir='keras_audio', target_path='keras_audio', overwrite=True, show_progress=False)
ws.get_default_datastore().upload(src_dir='model', target_path='model', overwrite=True, show_progress=False)

$AZUREML_DATAREFERENCE_9ccb1fda740147be9089bbc7febb5d89

In [59]:
# getting or creating a cluster

try:
    compute_target = ComputeTarget(workspace=ws, name=azureMLClusterName)
    print('using existing computer cluster: ' + azureMLClusterName)
except ComputeTargetException:
    print('Creating a new compute target: ' + azureMLClusterName)
    compute_config = AmlCompute.provisioning_configuration(vm_size=azureMLVMSize, vm_priority='lowpriority', max_nodes=azureMLMazNodes, 
        idle_seconds_before_scaledown=600, admin_username=azureMLClusterName[:8].lower(), admin_user_password=azureStorageKeyName[:20])
    compute_target = ComputeTarget.create(workspace=ws, name=azureMLClusterName, provisioning_configuration=compute_config)
    compute_target.wait_for_completion(show_output=True, timeout_in_minutes=20)

script_params = {
    '--data_dir': ds.as_mount()
}


using existing computer cluster: cluster3


In [60]:
# creating a trainer
est = TensorFlow(source_directory='.',
                 entry_script='run.py',
                 script_params=script_params,
                 compute_target=compute_target,
                 # sadly, the latest librosa does not support load, so ... loading version 0.6.0
                 pip_packages=['keras==2.2.5', 'sklearn', 'librosa==0.6.0', 'lru-dict==1.1.6', 'azure-storage-blob==0.37.1'],
                 use_gpu=("NC" in azureMLVMSize.upper()),
                 framework_version='1.13',
                 node_count=azureMLMazNodes,
                 process_count_per_node=1,
                 distributed_training=MpiConfiguration())

# running the trainer in an experiment
run = exp.submit(est)

# here we wait until completion so we can register the mode and download the files into this Jypyter Notebook
run.wait_for_completion(show_output=True)

model = run.register_model(model_name=experiment_name, 
                            model_path='outputs/model',
                            description=experiment_name + ' built on ' + str(azureMLMazNodes) + ' nodes of ' + azureMLVMSize + ' from ' + azureStorgeAccountName + '/' + azureStorageTargetContainer)

# downloading the files for the build step or pull it from the Azure Blob Storage: see run.py
run.download_file(name='outputs/model/cifar10-architecture.json', output_file_path ='../4-buildEdgeModules/audioinference/models/cifar10-architecture.json')
run.download_file(name='outputs/model/cifar10-config.npy', output_file_path ='../4-buildEdgeModules/audioinference/models/cifar10-config.npy')
run.download_file(name='outputs/model/cifar10-history.npy', output_file_path ='../4-buildEdgeModules/audioinference/models/cifar10-history.npy')
run.download_file(name='outputs/model/cifar10-weights.h5', output_file_path ='../4-buildEdgeModules/audioinference/models/cifar10-weights.h5')


RunId: KerasAudioExperiment_1593520866_0267cae6
Web View: https://ml.azure.com/experiments/KerasAudioExperiment/runs/KerasAudioExperiment_1593520866_0267cae6?wsid=/subscriptions/dc6f773e-4b13-4f8b-8d76-f34469246722/resourcegroups/audioRG/workspaces/audioWorkSpace
